In [1]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def rk5_cash_karp(x, y, dydx, dh, derivs):

    ai = array[0, 0.2, 0.3, 0.6, 1.0, 0.875]

    ci = [37.0/378.0, 0.0, 250.0/621.0, 125.0/594.0, 0.0, 512.0/1771.0]

    dci = [ci[0]-2825.0/27648.0, 0.0, ci[2]-18575.0/48384.0, ci[3]-13525.0/55296.0, -277.00/14336.0, ci[5]-0.25]
    
    bs = [[0.0, 0.0, 0.0, 0.0, 0.0, 0.0],
          [0.2, 0.0, 0.0, 0.0, 0.0, 0.0],
          [3.0/40.0, 9.0/40.0, 0.0, 0.0, 0.0, 0.0],
          [0.3, -0.9, 1.2, 0.0, 0.0, 0.0],
          [-11.0/54.0, 2.5, -70.0/27.0, 35.0/27.0, 0.0, 0.0],
          [1631.0/55296.0, 175.0/512.0, 575.0/13824.0, 44275.0/110592.0, 253.0/4096.0, 0.0]]

    k0 = dh*dydx # first step
    k1 = dh*derivs(y + bs[1,0]*k0, x + ai[1]*dh) # Second step.
    k2 = dh*derivs(y + bs[2,0]*k0+bs[2,1]*k1, x + ai[2]*dh) # Third step.
    k3 = dh*derivs(y + bs[3,0]*k0+bs[3,1]*k1+bs[3,2]*k2, x + ai[3]*dh) # Fourth step.
    k4 = dh*derivs(y + bs[4,0]*k0+bs[4,1]*k1+bs[4,2]*k2+bs[4,3]*k3, x + ai[4]*dh) # Fifth step.
    k5 = dh*derivs(y + bs[5,0]*k0+bs[5,1]*k1+bs[5,2]*k2+bs[5,3]*k3+bs[5,4]*k4, x + ai[5]*dh) # Sixth step.

    # Accumulate increments with proper weights.

    y_out = y + ci[0]*k0+ci[2]*k2+ci[3]*k3+ci[5]*k5

    # Estimate error as difference between fourth and fifth order methods.

    y_err = dci[0]*k0+dci[2]*k2+dci[3]*k3+dci[4]*k4+dci[5]*k5

    return (y_out, y_err)


In [3]:
def RK5_Step(x, y, dhtry, derivs, accuracy):

    dydx = derivs(y, x) # Calculates derivatives for the new step
    # good way of determining desired accuracy
    
    yscal = abs(y[:]) + abs(dydx[:]*dhtry) + 1e-3
    
    dh = dhtry # Set stepsize to the initial trial value.
    while True: # infinite loop
        
        (y_new, yerr) = RK5_Try(x, y, dydx, dh, derivs) # Take a step.
    
        errmax = max( abs(yerr/yscal) )/accuracy # maximum error scaled to required tolerance
            
        if (errmax <= 1.0): break # Step succeeded. Compute size of next step.
            
    dh_new = 0.9*dh/errmax**0.25 # Truncation error too large, reduce stepsize.
        
    if abs(dh_new) < 0.1*abs(dh): # if step might get too small
        dh_new = 0.1*dh # take at most 10-times smaller step
        dh = dh_new
        
        if ( x+dh == x): 
            print ("ERROR: stepsize underflow in RKStep")
        
        if errmax < 2.e-4: # Step is way too small
            dh_next = 5.0*dh # Increase it 5-times
            
        else: # Step was too small, but of correct order of magnitude
            dh_next = 0.9*dh/errmax**0.2 # Step is too small, increase it next time with the deltaˆ1/5. power
                
    return (x+dh, y_new, dh_next)

In [4]:
def rk5_mv(dydx,a,b,y_a,tol):
    #dydx is the derivative wrt x
    #a is lower bound/ b is upper bound
    #y_a is boundary conditions
    #tol: tolerance for int y
    
    #define starting step
    xi = a
    yi = y_a.copy()
    
    #an initial step size == make very small
    h= 1.0e-4*(b-a)
    
    imax=10000
    i=0
    
    #set # of coupled ode's to size y_a
    nv = len(y_a)
    
    #set initial conditions
    x = np.full(1,a)
    y = np.full((1,nv),y_a)
    
    #set a flag
    flag=1
    
    #loop til we reach right side
    while(flag):
        
        #calculate y_i+1
        yi_new, h_new, h_step = RK5_Step(x, y, dhtry, derivs, accuracy)
        
        #update new step
        h = h_new
        
        #prevent overshoot
        if(xi+h_step>b):
            
            #take smaller step
            h = b-xi
            
            #recalc y_i+1
            yi_new, h_new, h_step = RK5_Step(x, y, dhtry, derivs, accuracy)
            
            #break
            flag = 0
            
        #update values
        xi += h_step
        yi[:] = yi_new[:]
        
        # add step to arrays
        x = np.append(x,xi)
        y_new = np.zeros((len(x),nv))
        y_new[0:len(x)-1,:]=y
        y_new[-1,:]=yi[:]
        del y
        y = y_new
        
        #prevent too many itterations
        if(i>imax):
            
            print("Max iterations reached")
            raise StopIteration("Iteration number = ",i)
        
        #iterate
        i += 1
        
        #output some info
        s = "i = %3d\tx = %9.8f\th = %9.8f\tb=%9.8f" % (i,xi,h_step, b)
        print(s)
        
        #break if new xi is ==b
        if(xi==b):
            flag = 0
        
    #return answer
    return x,y

In [5]:
a = 0.0
b = 2.0*np.pi

y_0 = np.zeros(2)
y_0[0]=0.0
y_0[1]=1.0
nv=2

tolerance = 1.0e-6

#perform integration
x,y=rk5_mv(dydx,a,b,y_0,tolerance)

NameError: name 'dydx' is not defined

In [6]:
plt.plot(x,y[:,0],'o',label='y(x)')
plt.plot(x,y[:,1],'o',label='dydx(x)')
xx = np.linspace(0,2.0*np.pi,1000)
plt.plot(xx,np.sin(xx),label='sin(x)')
plt.plot(xx,np.cos(xx),label='cos(x)')
plt.xlabel('x')
plt.ylabel('y, dy/dx')
plt.legend(frameon=False)

NameError: name 'x' is not defined

In [7]:
sine = np.sin(x)
cosine = np.cos(x)

y_error = (y[:,0]-sine)
dydx_error = (y[:,1]-cosine)

plt.plot(x, y_error, label="y(x) Error")
plt.plot(x, dydx_error, label="dydx(x) Error")
plt.legend(frameon=False)

NameError: name 'x' is not defined